In [321]:
import nltk
import time
from collections import Counter
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

import csv
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize,blankline_tokenize
from nltk import PorterStemmer, LancasterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import re
import string
from collections import Counter
import json
import re as regex
import xgboost as xgb
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.metrics import r2_score


In [322]:
class TwitterData_Initialize():
    data = []
    processed_data = []
    wordlist = []

    featureList = []
    fea_vect=[]
    
    data_model = None
    data_labels = None
    is_testing = False
    
    def initialize(self, csv_file, is_testing_set=False, from_cached=None):
        if from_cached is not None:
            #self.data_model = pd.read_csv(from_cached)
            self.data_model = pd.read_json(from_cached)

            return

        self.is_testing = is_testing_set

        if not is_testing_set:
            #self.data = pd.read_csv(csv_file, header=0, names=["id", "emotion", "text"])
            self.data = pd.read_json(csv_file)

            #self.data = self.data[self.data["emotion"].isin(["positive", "negative", "neutral"])]
        

        self.processed_data = self.data
        self.wordlist = []
        self.data_model = None
        self.data_labels = None
        
    
    
    
    def do_process(self):
        start_time = time.time()
        def stem_and_join(row,stemmer=nltk.PorterStemmer()):
            row["spans"] = list(map(lambda str: stemmer.stem(str.lower()), row["spans"]))
            return row
    
        def tokenize_grams(row):
                
                # turn a doc into clean tokens
                def clean_doc(doc):
                    # split into tokens by white space
                    tokens = doc.split()
                    # remove punctuation from each token
                    table = str.maketrans('', '', string.punctuation)
                    tokens = [w.translate(table) for w in tokens]
                    # remove remaining tokens that are not alphabetic
                    tokens = [word for word in tokens if word.isalpha()]
                    # filter out stop words
                    #stop_words = set(stopwords.words('english'))
                    
                    #tokens = [w for w in tokens if not w in stop_words]
                    
                    
                    # filter out short tokens
                    tokens = [word.lower() for word in tokens if len(word) > 2]
                    return tokens
                




               
                # Function to apply lemmatization to a list of words
                def words_lemmatizer(words, encoding="utf8"):
                    wordnet_lemmatizer = WordNetLemmatizer()

                    lemma_words = []
                    wl = WordNetLemmatizer()
                    for word in words:
                        pos = find_pos(word)
                        lemma_words.append(wl.lemmatize(word, pos))
                    return lemma_words
                
                # Function to find part of speech tag for a word
                def find_pos(word):
                    # Part of Speech constants
                    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
   
                    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
                    # Adjective tags - 'JJ', 'JJR', 'JJS'
                    if pos.lower()[0] == 'j':
                        return 'a'
                    # Adverb tags - 'RB', 'RBR', 'RBS'
                    elif pos.lower()[0] == 'r':
                        return 'r'
                    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
                    elif pos.lower()[0] == 'v':
                        return 'v'
                    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
                    else:
                        return 'n'
                    
                    
                def get_ngrams(words,n):
                    n_grams = ngrams(words, n)
                    
                    list_grams=[ ' '.join(grams) for grams in n_grams]
                    return list_grams
    
            
                #convert to string
                idx=row["spans"]
                #ch="".join(x for x in idx if x)
                ch=' '.join(idx)
               
    
                
                
                n_grams=clean_doc(ch)
                tokens=words_lemmatizer(n_grams)
                bi_grams=get_ngrams(tokens,2)
                #print("nombre of tokens",len(n_grams))

                row["token_spans"] = tokens
                row["bi_grams"] = bi_grams
    
                return row
        #self.processed_data = self.processed_data.apply(stem_and_join, axis=1)
        self.processed_data = self.processed_data.apply(tokenize_grams, axis=1)
        print("--- %s seconds ---" % (time.time() - start_time))

    

    def build_wordlist(self, min_occurrences=0, max_occurences=500, stopwords=nltk.corpus.stopwords.words("english"),
                       ):
       
        self.wordlist = []
        whitelist=["to","on","for","up","below","short","long"]
        
        #stopwords=[]
       
        words = Counter()
        words_2=Counter()
        
        for idx in self.processed_data.index:
            words.update(self.processed_data.loc[idx, "token_spans"])

        #bigrams   
        for idx in self.processed_data.index:
            words_2.update(self.processed_data.loc[idx, "bi_grams"])
            
        for idx, stop_word in enumerate(stopwords):
            if stop_word not in whitelist:
                del words[stop_word]
                
                
        #bigrams        
        for idx, stop_word in enumerate(stopwords):
            if stop_word not in whitelist:
                del words_2[stop_word]
                
        
        
        print(words.most_common()+words_2.most_common())        
        
        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("wordlist_copie.csv", index_label="idx")
        self.wordlist = [k for k, v in (words.most_common()+words_2.most_common()) if min_occurrences < v < max_occurences]
        
        
        
    def build_data_model(self):
        label_column = []
        Id_column=["ID"]
        label_column = ["label"]

        columns = Id_column + label_column + list(
            map(lambda w: w ,self.wordlist))
        labels = []
        rows = []
        for idx in self.processed_data.index:
            current_row = []

            if True:
                # add label
                current_label = self.processed_data.loc[idx, "sentiment score"]
                current_id = self.processed_data.loc[idx, "id"]
                
                labels.append(current_id)
                labels.append(current_label)
                
                current_row.append(current_id)
                current_row.append(current_label)

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "token_spans"]+self.processed_data.loc[idx, "bi_grams"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        self.data_labels = pd.Series(labels)
        return self.data_model, self.data_labels


In [323]:
class TwitterData_Initialize_test(TwitterData_Initialize):

    
    
    def do_process(self):
        def stem_and_join(row,stemmer=nltk.PorterStemmer()):
            
            row["spans"] = list(map(lambda str: stemmer.stem(str.lower()), row["spans"]))
            return row
    
        def tokenize_grams(row):
                
                # Function to remove stop words
                def remove_stopwords(text, lang='english'):
                    whitelist = ["n't","not","below"]    

                    stop_words = set(stopwords.words('english'))
                    word_tokens = word_tokenize(text)
                    #filtered_sentence = [w for w in word_tokens if ((not w in stop_words) or (w in whitelist))]
                    filtered_sentence = []
                    for w in word_tokens:
                        if ((w not in stop_words) or (w in whitelist)):
                            filtered_sentence.append(w)
                
                    ch=" ".join(filtered_sentence)

                    return ch
#40

                def clean_doc(doc):
                    # split into tokens by white space
                    tokens = doc.split()
                    # remove punctuation from each token
                    table = str.maketrans('', '', string.punctuation)
                    tokens = [w.translate(table) for w in tokens]
                    # remove remaining tokens that are not alphabetic
                    tokens = [word for word in tokens if word.isalpha()]
                    # filter out stop words
                    #stop_words = set(stopwords.words('english'))
                    #tokens = [w for w in tokens if not w in stop_words]
                    
                    # filter out short tokens
                    tokens = [word.lower() for word in tokens if len(word) > 2]
                    return tokens


                

                # Function to apply lemmatization to a list of words
                def words_lemmatizer(words, encoding="utf8"):
                    wordnet_lemmatizer = WordNetLemmatizer()

                    lemma_words = []
                    wl = WordNetLemmatizer()
                    for word in words:
                        pos = find_pos(word)
                        lemma_words.append(wl.lemmatize(word, pos))
                    return lemma_words

                # Function to find part of speech tag for a word
                def find_pos(word):
                    # Part of Speech constants
                    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
   
                    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
                    # Adjective tags - 'JJ', 'JJR', 'JJS'
                    if pos.lower()[0] == 'j':
                        return 'a'
                    # Adverb tags - 'RB', 'RBR', 'RBS'
                    elif pos.lower()[0] == 'r':
                        return 'r'
                    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
                    elif pos.lower()[0] == 'v':
                        return 'v'
                    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
                    else:
                        return 'n'
                    
                def get_ngrams(words,n):
                    n_grams = ngrams(words, n)
                    
                    list_grams=[ ' '.join(grams) for grams in n_grams]
                    return list_grams
    
            
                token=[]
                idx=row["spans"]
                ch=' '.join(idx)


    
                #words_stemmer(ch, type="PorterStemmer", lang="english", encoding="utf8")
                #Convert to lower case
                n_grams=clean_doc(ch)
                tokens=words_lemmatizer(n_grams)
                bi_grams=get_ngrams(tokens,2)

                #print("list ngrams--------------\n",n_grams)
                row["token_spans"] = tokens
                row["bi_grams"] = bi_grams
    
                return row
        #self.processed_data = self.processed_data.apply(stem_and_join, axis=1)
        self.processed_data = self.processed_data.apply(tokenize_grams, axis=1)
        
    def build_wordlist(self, min_occurrences=0, max_occurences=500, stopwords=nltk.corpus.stopwords.words("english"),
                      ):
        
        whitelist=["to","on","for","up","below","short","long"]
        self.wordlist = []
        #stopwords=[]
        
        
        words = Counter()
        words_2=Counter()
        
        for idx in self.processed_data.index:
            words.update(self.processed_data.loc[idx, "token_spans"])

        #bigrams   
        for idx in self.processed_data.index:
            words_2.update(self.processed_data.loc[idx, "bi_grams"])
            
        for idx, stop_word in enumerate(stopwords):
            if stop_word not in whitelist:
                del words[stop_word]
                
                
        #bigrams        
        for idx, stop_word in enumerate(stopwords):
            if stop_word not in whitelist:
                del words_2[stop_word]
                
        
        
        print(words.most_common())        
        print(words_2.most_common())        

        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("wordlist_test.csv", index_label="idx")
        self.wordlist = [k for k, v in (words.most_common()+words_2.most_common()) if min_occurrences < v < max_occurences]
        
        
    def build_data_model(self):
        
        label_id = ["ID"]

        columns = label_id + list(
            map(lambda w: w ,self.wordlist))
        labels = []
        rows = []
        for idx in self.processed_data.index:
            current_row = []
            if True:
                # add label
                current_id = self.processed_data.loc[idx, "id"]
                
                labels.append(current_id)
                
                current_row.append(current_id)

            

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "token_spans"]+self.processed_data.loc[idx, "bi_grams"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        return self.data_model


In [324]:
data = TwitterData_Initialize()
data.initialize("Microblog_Trainingdata.json")

data.processed_data.head(20)

,cashtag,id,sentiment score,source,spans
0,$FB,719659409228451840,0.366,twitter,[watching for bounce tomorrow]
1,$LUV,719904304207962112,0.638,twitter,[record number of passengers served in 2015]
2,$NFLX,5329774,-0.494,stocktwits,[out $NFLX -.35]
3,$DIA,719891468173844480,0.460,twitter,"[Looking for a strong bounce, Lunchtime rally ..."
4,$PLUG,20091246,0.403,stocktwits,[Very intrigued with the technology and growth...
5,$GMCR,5819749,0.000,stocktwits,"[short worked, puts up]"
6,$IBM,709741154393133056,-0.296,twitter,[overbought]
7,$JOSB,17892972,-0.546,stocktwits,"[absolute garbage still up, stores TOTALLY EMP..."
8,$CSTM,709834259687710720,-0.438,twitter,[Biggest Market Losers]
9,$PYPL,708481442079068160,0.408,twitter,[Love this company long time.]


In [325]:
data_test = TwitterData_Initialize_test()
data_test.initialize("Microblog_Trialdata.json")

data_test.processed_data.head()
test_data=data_test.processed_data.copy()

In [326]:
data.do_process()

C:\Users\Globalnet\Anaconda3\lib\site-packages\ipykernel_launcher.py:103: DeprecationWarning: generator 'ngrams' raised StopIteration


--- 233.88764786720276 seconds ---


In [327]:
data.processed_data.head(20)
tokens=data.processed_data["token_spans"]
taille=0
for token in tokens:
    taille+=len(token)
print(taille)



8314


In [328]:
data.processed_data.head(20)


,cashtag,id,sentiment score,source,spans,token_spans,bi_grams
0,$FB,719659409228451840,0.366,twitter,[watching for bounce tomorrow],"[watch, for, bounce, tomorrow]","[watch for, for bounce, bounce tomorrow]"
1,$LUV,719904304207962112,0.638,twitter,[record number of passengers served in 2015],"[record, number, passenger, serve]","[record number, number passenger, passenger se..."
2,$NFLX,5329774,-0.494,stocktwits,[out $NFLX -.35],"[out, nflx]",[out nflx]
3,$DIA,719891468173844480,0.460,twitter,"[Looking for a strong bounce, Lunchtime rally ...","[look, for, strong, bounce, lunchtime, rally, ...","[look for, for strong, strong bounce, bounce l..."
4,$PLUG,20091246,0.403,stocktwits,[Very intrigued with the technology and growth...,"[very, intrigue, with, the, technology, and, g...","[very intrigue, intrigue with, with the, the t..."
5,$GMCR,5819749,0.000,stocktwits,"[short worked, puts up]","[short, work, put]","[short work, work put]"
6,$IBM,709741154393133056,-0.296,twitter,[overbought],[overbought],[]
7,$JOSB,17892972,-0.546,stocktwits,"[absolute garbage still up, stores TOTALLY EMP...","[absolute, garbage, still, store, totally, emp...","[absolute garbage, garbage still, still store,..."
8,$CSTM,709834259687710720,-0.438,twitter,[Biggest Market Losers],"[big, market, loser]","[big market, market loser]"
9,$PYPL,708481442079068160,0.408,twitter,[Love this company long time.],"[love, this, company, long, time]","[love this, this company, company long, long t..."


In [329]:

data.build_wordlist()
data.wordlist.sort()
def save_list(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()
    
save_list(data.wordlist,"list_word_copie.txt")
len(data.wordlist)

[('for', 128), ('stock', 126), ('short', 95), ('look', 83), ('buy', 76), ('long', 74), ('today', 68), ('sell', 66), ('high', 66), ('call', 59), ('market', 49), ('good', 47), ('nice', 44), ('new', 44), ('still', 43), ('get', 43), ('day', 43), ('add', 42), ('week', 38), ('top', 38), ('bullish', 37), ('like', 33), ('put', 32), ('close', 31), ('big', 30), ('rally', 29), ('hold', 29), ('low', 29), ('trade', 28), ('go', 28), ('take', 28), ('move', 27), ('break', 26), ('breakout', 26), ('strong', 25), ('tsla', 25), ('setup', 25), ('price', 25), ('buying', 24), ('see', 24), ('play', 24), ('aapl', 24), ('run', 23), ('support', 23), ('back', 23), ('start', 23), ('come', 22), ('time', 22), ('could', 22), ('downgrade', 22), ('weak', 21), ('lead', 21), ('bad', 20), ('one', 20), ('bull', 20), ('make', 20), ('growth', 19), ('overbought', 19), ('may', 19), ('share', 19), ('year', 19), ('next', 18), ('great', 18), ('sale', 18), ('well', 18), ('sector', 18), ('gain', 18), ('bought', 18), ('earnings', 17

6453

In [330]:
bow, labels = data.build_data_model()
bow.head(20)


,ID,label,aal,aal luv,aapl,aapl and,aapl cleary,aapl could,aapl cult,aapl drag,...,zombie,zombie stock,zone,zone bot,zone strong,zone today,zuck,zuck told,zuckerberg,zuckerberg do
0,719659409228451840,0.366,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,719904304207962112,0.638,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5329774,-0.494,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,719891468173844480,0.460,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20091246,0.403,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5819749,0.000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,709741154393133056,-0.296,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,17892972,-0.546,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,709834259687710720,-0.438,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,708481442079068160,0.408,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [331]:
len(bow.columns)

6455

In [332]:
data_test.do_process()
data_test.processed_data.head(20)



,cashtag,id,sentiment score,source,spans,token_spans,bi_grams
0,$F,5540055,-0.454,stocktwits,[Putting on a little $F short],"[put, little, short]","[put little, little short]"
1,$AAPL,10752226,-0.464,stocktwits,[short some],"[short, some]",[short some]
2,$BAC,10920221,0.445,stocktwits,[buying opportunity],"[buying, opportunity]",[buying opportunity]
3,$SHOR,12971398,0.661,stocktwits,[Scaling Up on Long Position],"[scale, long, position]","[scale long, long position]"
4,$JPM,16142438,-0.763,stocktwits,[its time to sell banks],"[it, time, sell, bank]","[it time, time sell, sell bank]"
5,$LMT,14073133,0.627,stocktwits,[Entering long],"[enter, long]",[enter long]
6,$DNN,18479024,0.653,stocktwits,[picked some up],"[picked, some]",[picked some]
7,$CRK,34147106,0.668,stocktwits,"[time to accumulate for a long position, far m...","[time, accumulate, for, long, position, far, m...","[time accumulate, accumulate for, for long, lo..."
8,$CRK,34147107,0.460,stocktwits,"[Looking for a strong bounce, Lunchtime rally ...","[look, for, strong, bounce, lunchtime, rally, ...","[look for, for strong, strong bounce, bounce l..."
9,$CRK,34147108,0.403,stocktwits,[Very intrigued with the technology and growth...,"[very, intrigue, with, the, technology, and, g...","[very intrigue, intrigue with, with the, the t..."


In [333]:
data_test.build_wordlist()
data_test.wordlist.sort()

print(len(data_test.wordlist))
bow_test= data_test.build_data_model()
bow_test.head(20)

[('short', 4), ('put', 3), ('long', 3), ('little', 2), ('buying', 2), ('position', 2), ('time', 2), ('for', 2), ('opportunity', 1), ('scale', 1), ('sell', 1), ('bank', 1), ('enter', 1), ('picked', 1), ('accumulate', 1), ('far', 1), ('upside', 1), ('downside', 1), ('look', 1), ('strong', 1), ('bounce', 1), ('lunchtime', 1), ('rally', 1), ('come', 1), ('intrigue', 1), ('technology', 1), ('growth', 1), ('potential', 1), ('work', 1), ('big', 1), ('market', 1), ('loser', 1), ('goog', 1), ('googl', 1), ('would', 1), ('suck', 1), ('sbux', 1), ('dip', 1), ('below', 1), ('overbought', 1), ('dont', 1)]
[('little short', 2), ('long position', 2), ('put little', 1), ('short some', 1), ('buying opportunity', 1), ('scale long', 1), ('it time', 1), ('time sell', 1), ('sell bank', 1), ('enter long', 1), ('picked some', 1), ('time accumulate', 1), ('accumulate for', 1), ('for long', 1), ('position far', 1), ('far more', 1), ('more upside', 1), ('upside than', 1), ('than downside', 1), ('look for', 1), 

,ID,accumulate,accumulate for,and growth,and overbought,bank,below,below and,big,big market,...,time accumulate,time sell,upside,upside than,very intrigue,with the,work,work put,would,would suck
0,5540055,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10752226,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10920221,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,12971398,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,16142438,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5,14073133,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,18479024,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,34147106,1,1,0,0,0,0,0,0,0,...,1,0,1,1,0,0,0,0,0,0
8,34147107,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,34147108,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0


In [334]:
bow_test.columns

Index(['ID', 'accumulate', 'accumulate for', 'and growth', 'and overbought',
       'bank', 'below', 'below and', 'big', 'big market', 'bounce',
       'bounce lunchtime', 'buying', 'buying opportunity', 'buying sbux',
       'come', 'dip', 'dont', 'dont put', 'down little', 'downside', 'enter',
       'enter long', 'far', 'far more', 'for', 'for long', 'for strong',
       'goog', 'goog googl', 'googl', 'googl would', 'growth',
       'growth potential', 'intrigue', 'intrigue with', 'it time', 'little',
       'little short', 'long', 'long position', 'look', 'look for', 'loser',
       'lunchtime', 'lunchtime rally', 'market', 'market loser', 'more upside',
       'opportunity', 'overbought', 'picked', 'picked some', 'position',
       'position far', 'potential', 'put', 'put down', 'put little', 'rally',
       'rally come', 'sbux', 'sbux dip', 'scale', 'scale long', 'sell',
       'sell bank', 'short', 'short below', 'short some', 'strong',
       'strong bounce', 'suck', 'technolog

In [335]:
#data.wordlist.index("fit")

In [336]:

"""
#convert bow dataframe to numpy matrix

Y = bow.label.as_matrix()
X_drop = bow.drop({"label","ID"},axis=1)

X=X_drop.as_matrix()
"""

'\n#convert bow dataframe to numpy matrix\n\nY = bow.label.as_matrix()\nX_drop = bow.drop({"label","ID"},axis=1)\n\nX=X_drop.as_matrix()\n'

In [337]:

for word in data.wordlist:
    current_row=[]
    if (not word  in data_test.wordlist):
        # add label
        for idx in bow_test.index:
            current_row.append(0)
        bow_test[word]=current_row

    
bow_test.head(20)


,ID,accumulate,accumulate for,and growth,and overbought,bank,below,below and,big,big market,...,zombie,zombie stock,zone,zone bot,zone strong,zone today,zuck,zuck told,zuckerberg,zuckerberg do
0,5540055,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10752226,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10920221,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,12971398,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,16142438,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14073133,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,18479024,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,34147106,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,34147107,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,34147108,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [338]:
print(len(bow_test.columns))
#bow_test.sort_index(axis = 1, ascending = False)
#bow.sort_index(axis = 1, ascending = True)


6468


In [339]:

#bow = bow.rename(columns = {'fit': 'fit_'})

#X_train, X_test, y_train, y_test = train_test_split(bow.iloc[:,2:], bow.iloc[:,1])                                                  
#id_vals=X_test.index
y_train=bow["label"]
X_train=bow.drop({"ID","label"},axis=1)


In [340]:
"""CV_time = time.time()
# A parameter grid for XGBoost
params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}

# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=-1) 
grid = GridSearchCV(xgb, params)
grid.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - CV_time))
"""

'CV_time = time.time()\n# A parameter grid for XGBoost\nparams = {\'min_child_weight\':[4,5], \'gamma\':[i/10.0 for i in range(3,6)],  \'subsample\':[i/10.0 for i in range(6,11)],\n\'colsample_bytree\':[i/10.0 for i in range(6,11)], \'max_depth\': [2,3,4]}\n\n# Initialize XGB and GridSearch\nxgb = XGBRegressor(nthread=-1) \ngrid = GridSearchCV(xgb, params)\ngrid.fit(X_train, y_train)\nprint("--- %s seconds ---" % (time.time() - CV_time))\n'

In [341]:
"""grid.best_params_"""

'grid.best_params_'

In [342]:
#Fitting XGB regressor with parameters obtained by Grid searchCV
params={'colsample_bytree': 1.0,
 'gamma': 0.3,
 'max_depth': 4,
 'min_child_weight': 4,
 'subsample': 0.8}
model = xgb.XGBRegressor(max_depth=4, min_child_weight= 4,gamma=0.3,subsample=0.8,colsample_bytree=1.0)

model.fit(X_train,y_train)
print (model)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1.0, gamma=0.3, learning_rate=0.1,
       max_delta_step=0, max_depth=4, min_child_weight=4, missing=None,
       n_estimators=100, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.8)


In [343]:
#test model accuracy
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


print("mean squared error:" ,mean_squared_error(model.predict(X_train), y_train))
print("R2 score:" ,r2_score(model.predict(X_train),y_train))

mean squared error: 0.0773110592283901
R2 score: -1.1728767696723867


In [344]:
"""
X_test=bow_test.drop({"ID"},axis=1)
X_test = X_test.as_matrix()

y_test=data_test.processed_data["sentiment score"]
id_vals=y_test.index
"""

'\nX_test=bow_test.drop({"ID"},axis=1)\nX_test = X_test.as_matrix()\n\ny_test=data_test.processed_data["sentiment score"]\nid_vals=y_test.index\n'

In [345]:

#reorganize test columns as the train columns

#X_test = bow_test.as_matrix()

#X_test

bow_test=bow_test[bow.drop(["label"],axis=1).columns]
X_test=bow_test.drop({"ID"},axis=1)
y_test=data_test.processed_data["sentiment score"]
id_vals=X_test.index


In [346]:

bow_test.columns
bow_test.head()
col=list(bow_test.columns)
col.index("bank")


459

In [347]:
#if train_test_split used
"""tweet_row=[]
labels=["ID","tweet"]
for num in id_vals:
    current_row=[]
    current_id = bow.loc[num, "ID"]
                
    current_tweet=data.processed_data.loc[data.processed_data['id'] == current_id,"spans"].iloc[0]

                
    current_row.append(current_id)
    current_row.append(current_tweet)
    

   

    tweet_row.append(current_row) 
tweet_row"""

'tweet_row=[]\nlabels=["ID","tweet"]\nfor num in id_vals:\n    current_row=[]\n    current_id = bow.loc[num, "ID"]\n                \n    current_tweet=data.processed_data.loc[data.processed_data[\'id\'] == current_id,"spans"].iloc[0]\n\n                \n    current_row.append(current_id)\n    current_row.append(current_tweet)\n    \n\n   \n\n    tweet_row.append(current_row) \ntweet_row'

In [348]:
#Predict 
#output=grid.best_estimator_.predict(X_test)
tweet_row=list(data_test.processed_data["spans"])
old_pred=list(data_test.processed_data["sentiment score"])
output = model.predict(X_test)
#final_df = pd.DataFrame(tweet_row, columns=labels)
final_df = pd.DataFrame()
final_df["tweet"] = tweet_row

final_df["old_pred"] = old_pred

final_df["Prediction"] = output
final_df.to_csv("Output_1.csv",sep=",")
final_df.head(20)

,tweet,old_pred,Prediction
0,[Putting on a little $F short],-0.454,-0.420749
1,[short some],-0.464,-0.264636
2,[buying opportunity],0.445,0.170020
3,[Scaling Up on Long Position],0.661,0.442809
4,[its time to sell banks],-0.763,-0.240355
5,[Entering long],0.627,0.349393
6,[picked some up],0.653,0.152160
7,"[time to accumulate for a long position, far m...",0.668,0.468260
8,"[Looking for a strong bounce, Lunchtime rally ...",0.460,0.425000
9,[Very intrigued with the technology and growth...,0.403,0.223042


In [349]:
#test model accuracy
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


print("mean squared error:" ,mean_squared_error(output, y_test))
print("R2 score:" ,r2_score(output,y_test))

mean squared error: 0.0717591491975677
R2 score: 0.2759856696413664
